In [4]:
import pandas as pd
import numpy as np
import random
from datetime import datetime, timedelta

# Load the existing dataset
df = pd.read_excel('HR_Prompt_Dataset.xlsx', engine='openpyxl')

# Function to generate random dates excluding weekends
def generate_dates(start_date, end_date, exclude_weekends=True):
    dates = pd.date_range(start_date, end_date, freq='B' if exclude_weekends else 'D')
    return dates

# Function to generate random final DOJ dates (Tuesdays and Thursdays)
def generate_final_doj(start_date, end_date):
    dates = pd.date_range(start_date, end_date, freq='W-TUE').append(pd.date_range(start_date, end_date, freq='W-THU'))
    return dates

# Function to calculate notice days and bucket
def calculate_notice_days_and_bucket(offer_date, doj):
    notice_days = (doj - offer_date).days
    if 5 <= notice_days <= 15:
        bucket = 'Less than 15 days'
    elif 16 <= notice_days <= 30:
        bucket = '16 - 30 days'
    elif 31 <= notice_days <= 60:
        bucket = '31 - 60 days'
    elif 61 <= notice_days <= 90:
        bucket = '61 - 90 days'
    else:
        bucket = 'Greater than 90 days'
    return notice_days, bucket

# Function to generate random CTC values based on grade
def generate_ctc(grade):
    grade_ctc_ranges = {
        'P1': (500000, 1000000),
        'P2': (1000000, 1800000),
        'P3': (1500000, 2700000),
        'P4': (2400000, 3800000),
        'P5': (3200000, 4800000),
        'P6': (4000000, 5500000)
    }
    return random.randint(*grade_ctc_ranges[grade])

# Function to generate offered CTC based on current CTC
def generate_offered_ctc(current_ctc):
    return current_ctc + random.randint(int(current_ctc * 0.1), int(current_ctc * 0.7))

# Function to generate deviation percentage and band
def generate_deviation_percentage_and_band():
    deviation_percentage = random.uniform(-60, 95)
    if deviation_percentage <= 50:
        band = 'Less than or equal to P50'
    elif deviation_percentage <= 75:
        band = 'P50-P75'
    elif deviation_percentage <= 90:
        band = 'P75-P90'
    else:
        band = 'Greater than P90'
    return deviation_percentage, band

# Function to map C&B inputs to deviation percentile band C&B
def map_cb_inputs_to_band(cb_input):
    mapping = {
        'Between P50 (Median) to P75': 'P50-P75',
        'Beyond P90 (Max)': 'Greater than P90',
        'Upto P50 (Median)': 'Less than or equal to P50',
        'Between P75 to P90 (Max)': 'P75-P90',
        'Not applicable': random.choice(['P75-P90', 'P75 to P90 (Max)'])
    }
    return mapping[cb_input]

# Function to map Mapped_PU to Mapped_PU_Group and Mapped_PU_Group1
def map_pu_to_group(pu):
    pu_group_mapping = {
        'AS': ('Asia Pacific and Japan', 'APJ'),
        'BFS': ('BFS', 'BFS'),
        'CIS': ('Cloud and Infra', 'CIS'),
        'CG': ('Cuelogic', 'CG'),
        'SEC': ('Cyber Security', 'SEC'),
        'DATA': ('Data and Analytics', 'DATA'),
        'DL': ('Digital Engg', 'DGTE'),
        'GV': ('Enterprise AI', 'GV'),
        'Enterprise Cloud Apps': ('Enterprise Cloud Apps', 'Enterprise Cloud Apps'),
        'NC': ('Insurance', 'INS'),
        'ILP': ('Interactive and Low Code Platforms', 'ILP'),
        'IIOT': ('iNXT', 'IIOT'),
        'FT': ('Microsoft', 'Microsoft'),
        'OR': ('Oracle', 'OR'),
        'PN': ('Platform Operations', 'PN'),
        'TT': ('Quality Engineering', 'TT'),
        'RCG': ('Retail and Consumer Packaged Goods', 'RCG'),
        'SAP': ('SAP', 'SAP')
    }
    return pu_group_mapping[pu]

# Function to map Mapped_PU_Group1 to Mapped_Verticals and Mapped_SBU
def map_group1_to_verticals(group1):
    verticals_mapping = {
        'APJ': 'APJ',
        'BFS': 'BFS',
        'CIS': 'BFS,MFG,HI-Tech',
        'CG': 'Practice Specific',
        'SEC': 'Practice Specific',
        'DATA': 'Practice Specific, RCG, TTH',
        'DGTE': 'HI-Tech,DL,BFS',
        'GV': 'Practice Specific',
        'Enterprise Cloud Apps': 'Utilities, RCG, TTH',
        'INS': 'INS',
        'ILP': 'MFG, IME, Practice Specific',
        'IIOT': 'Practice Specific',
        'Microsoft': 'Microsoft',
        'OR': 'INS, TTH, RCG, MFG',
        'PN': 'Practice Specific, RCG, TTH',
        'TT': 'BFS, INS, TTH, RCG, MFG',
        'RCG': 'RCG',
        'SAP': 'M&E, RCG, HI-Tech'
    }
    return verticals_mapping[group1]

# Generate new records
new_records = []
start_date = datetime(2023, 1, 1)
end_date = datetime(2024, 8, 31)
offer_dates = generate_dates(start_date, end_date)
final_doj_dates = generate_final_doj(start_date, end_date)

for _ in range(700000 - len(df)):
    offer_date = random.choice(offer_dates)
    final_doj = random.choice(final_doj_dates)
    
    candidate_id = f"ID{random.randint(10000,99999)}"
    
    grade_classification = random.choice(['P1', 'P2', 'P3', 'P4', 'P5', 'P6'])
    
    current_ctc = generate_ctc(grade_classification)
    
    offered_ctc = generate_offered_ctc(current_ctc)
    
    deviation_percentage, deviation_band = generate_deviation_percentage_and_band()
    
    cb_input = random.choice(['Between P50 (Median) to P75', 
                              'Beyond P90 (Max)', 
                              'Upto P50 (Median)', 
                              'Between P75 to P90 (Max)', 
                              'Not applicable'])
    
    cb_band = map_cb_inputs_to_band(cb_input)
    
    mapped_pu = random.choice(['SAP', 'DL', 'PN', 'AS', 'DATA', 'Enterprise Cloud Apps', 'CIS', 'RE', 'OR', 'ILP', 'CG', 'BFS', 'TT', 'FT', 'SEC', 'GV', 'IIOT', 'NC'])
    
    mapped_pu_group, mapped_pu_group1 = map_pu_to_group(mapped_pu)
    
    mapped_customer_group = "Masked Company" # Placeholder for masked company names
    
    reactive_proactive = random.choice(['Reactive', 'Proactive'])
    
    mapped_verticals_sbu = map_group1_to_verticals(mapped_pu_group1)
    
    du_own_practice = random.choice(['Practice', 'DU owned'])
    
    joining_status = "Joined"
    
    doj_quarter_fy = f"Q{((final_doj.month-1)//3)+1} FY {final_doj.year%100}-{(final_doj.year+1)%100}"
    
    notice_days, notice_bucket = calculate_notice_days_and_bucket(offer_date, final_doj)
    
    new_record = {
        'Candidate_ID': candidate_id,
        'Offer_Created_Date': offer_date,
        'Final_DOJ': final_doj,
        'Notice_Days': notice_days,
        'Notice_Period_Bucket': notice_bucket,
        'Grade_Classification': grade_classification,
        'Grade_Harmonized': grade_classification,
        'Final_DOJ_Month': final_doj.strftime('%b-%y'),
        'Current_CTC': current_ctc,
        'Offered_CTC': offered_ctc,
        'Deviation_Percentage': deviation_percentage,
        'Deviation_Percentile_Band': deviation_band,
        'C_and_B_Inputs': cb_input,
        'Deviation_Percentile_Band_C_and_B': cb_band,
        'Mapped_PU': mapped_pu,
        'Mapped_PU_Group': mapped_pu_group,
        'Mapped_PU_Group1': mapped_pu_group1,
        'Mapped_Customer_Group': mapped_customer_group,
        'Reactive_and_Proactive': reactive_proactive,
        'Mapped_Verticals': mapped_verticals_sbu,
        'Mapped_SBU': mapped_verticals_sbu,
        'DU_Own_and_Practice': du_own_practice,
        'Joining_Status': joining_status,
        '

SyntaxError: EOL while scanning string literal (587697036.py, line 193)